In [1]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 7.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.6 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
"""데이터세트 다운로드 및 전처리"""
"""즉, 토큰화된 데이터를 바탕으로 언어별 어휘사전을 구축함."""

from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#텍스트 데이터를 토큰화하는 함수
def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE : 0, TGT_LANGUAGE : 1}

    for text in text_iter:
        yield token_transform[language](text[language_index[language]]) #해당 언어의 텍스트를 받아서 해당언어의 토크나이저로 토큰화
        #return과 비슷한 느낌인데 하나씩 생성하고 반환(값을 저장)

SRC_LANGUAGE = "de" #소스 언어
TGT_LANGUAGE = "en" #타겟 언어
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0,1,2,3
special_symbols = ["<unk>","<pad>","<bos>","<eos>"]

token_transform = {
    #spacy 라이브러리로 말뭉치를 토큰화하는 사전 학습된 모델
    SRC_LANGUAGE : get_tokenizer("spacy", language="de_core_news_sm"), #독일어 말뭉치
    TGT_LANGUAGE : get_tokenizer("spacy", language="en_core_web_sm"), #영어 말뭉치
    }
print("Token Transform:")
print(token_transform)

vocab_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = Multi30k(split="train", language_pair=(SRC_LANGUAGE,TGT_LANGUAGE))
    #학습 데이터 불러옴. (독일어 문장, 해당하는 영어 변역문)
    vocab_transform[language] = build_vocab_from_iterator( #언어별 어휘사전 생성
        generate_tokens(train_iter, language),
        min_freq=1, #어휘 사전에 포함시킬 최소 빈도수
        specials=special_symbols, #어휘 사전에 포함될 특수 토큰 지정
        special_first=True, #특수 토큰을 어휘 사전 맨 앞에 저장
    )

#모르는 토큰들에 사용될 인덱스 0 설정
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

print("Vocab Transform:")
print(vocab_transform)

Token Transform:
{'de': functools.partial(<function _spacy_tokenize at 0x159810220>, spacy=<spacy.lang.de.German object at 0x287a109d0>), 'en': functools.partial(<function _spacy_tokenize at 0x159810220>, spacy=<spacy.lang.en.English object at 0x287c1b8d0>)}
Vocab Transform:
{'de': Vocab(), 'en': Vocab()}


In [3]:
"""트랜스포머 모델 구성"""

import math
import torch
from torch import nn

#위치 인코딩
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout) #과적합 방지를 위해 드롭아웃

        position = torch.arange(max_len).unsqueeze(1)
        div_term =torch.exp(
            torch.arange(0,d_model,2)*(-math.log(10000.0)/d_model) #d_model/2 개의 텐서에 스케일링 인자를 곱해서 각기 다른값을 만들어준다.
            #(maxlen*(d_model/2))크기의 행렬이 생긴다고도 직관적으로 이해할 수 있을 듯
        )

        pe = torch.zeros(max_len, 1, d_model) 
        pe[:,0,0::2] = torch.sin(position*div_term)
        pe[:,0,1::2] = torch.cos(position*div_term)
        self.register_buffer("pe",pe)  #모델이 매개변수를 갱신하지 않도록 설정

    def forward(self,x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
#토큰 임베딩
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size) #d_model과 embsize는 같은 값임
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long())*math.sqrt(self.emb_size) 
               #입력된 토큰을 정수로 변환(long)하여 임베딩 벡터를 조회한다. 이때 임베딩 벡터는 1*d_model 사이즈
               #임베딩 벡터에 d_model 의 제곱근을 곱해서 크기 조정->학습 안정성 증진(임베딩 차원이 클수록 개별 차원에 할당되는 분산이 작아지므로, 이를 조정하기 위해 값을 곱하고 전체 벡터의 분산을 유지)

#TokenEmbedding 클래스로 소스데이터와 입력데이터를 입력 임베딩으로 변환하여 src_tok_emb, tgt_tok_emb 생성
class Seq2SeqTransformer(nn.Module):
    def __init__(
            self,
            num_encoder_layers,
            num_decoder_layers,
            emb_size,
            max_len,
            nhead,
            src_vocab_size,
            tgt_vocab_size,
            dim_feedforward,
            dropout = 0.1
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            d_model=emb_size, max_len=max_len, dropout=dropout
        )
        self.transformer = nn.Transformer(
            d_model=emb_size, #트랜스포머 모델의 입력과 출력 차원을 정의
            nhead=nhead, #멀티 헤드 어텐션의 개수
            num_encoder_layers=num_encoder_layers, #인코더 계층 개수
            num_decoder_layers=num_decoder_layers, #디코더 계층 계수
            dim_feedforward=dim_feedforward, #순방향 신경망의 은닉층 크기
            dropout=dropout, #인코더와 디코더에 적용되는 드롭아웃 비율
        )
        self.generator = nn.Linear(emb_size,tgt_vocab_size) 
        #디코더의 출력을 받아 로짓 생성으로 입력 사이즈는 emb_size, 출력 사이즈는 tgt_vocab_size

    def forward(
            self,
            src, #[소스 시퀀스 길이, 배치 크기, 임베딩 차원]
            trg, #[타겟 시퀀스 길이, 배치 크기, 임베딩 차원]
            src_mask, #[소스 시퀀스 길이, 시퀀스 길이]
            tgt_mask, #[타깃 시퀀스 길이, 시퀀스 길이]
            src_padding_mask,
            tgt_padding_mask,
            memory_key_padding_mask, #패딩 토큰이 위치한 부분을 가리는(0으로 만드는) 마스크
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.generator(outs) #어휘 사전에 대한 로짓 생성
    
    def encode(self,src,src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)),src_mask
        )
    
    def decode(self,tgt,memory,tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)),memory,tgt_mask
        )

In [8]:
"""트랜스포머 모델 구조"""

from torch import optim

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = Seq2SeqTransformer(
    num_encoder_layers=3,
    num_decoder_layers=3,
    emb_size=512,
    max_len=512,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512
).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = optim.Adam(model.parameters())

for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└",sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └",sssub_name)

src_tok_emb
└ embedding
tgt_tok_emb
└ embedding
positional_encoding
└ dropout
transformer
└ encoder
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  └ norm
└ decoder
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  └ norm
generator


/Users/yeeun/anaconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
"""배치 데이터 생성"""


from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

#여러 개의 전처리 함수를 인자로 받아 이를 차례로 적용하는 함수를 반환하는 함수
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

#인덱스화된 토큰에 특수토큰 할당
def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

#배치 단위로 데이터를 처리
def collator(batch):
    src_batch, tgt_batch = [],[]
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX) #소스 시퀀스 패딩
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX) #타겟 시퀀스 패딩
    return src_batch, tgt_batch


text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    ) # 문장을 토큰화, 토큰을 인덱스화, 인덱스화된 토큰에 특수토큰 할당

data_iter = Multi30k(split="valid", language_pair=(SRC_LANGUAGE,TGT_LANGUAGE)) #데이터세트 불러옴
dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator) #데이터세트를 데이터로더에 적용
source_tensor, target_tensor = next(iter(dataloader))

print("(source, target) :")
print(next(iter(data_iter)))

print("source_batch : ", source_tensor.shape) #[소스 시퀀스 길이, 배치 크기]
print(source_tensor)

print("target_batch : ", target_tensor.shape) #[타깃 시퀀스 길이, 배치 크기]
print(target_tensor)

(source, target) :
('Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen', 'A group of men are loading cotton onto a truck')
source_batch :  torch.Size([35, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  14,    5,    5,  ...,    5,   21,    5],
        [  38,   12,   35,  ...,   12, 1750,   69],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch :  torch.Size([30, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   6,    6,    6,  ...,  250,   19,    6],
        [  39,   12,   35,  ...,   12, 3254,   61],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


/Users/yeeun/anaconda3/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [10]:
"""어텐션 마스크 생성"""

def generate_square_subsequent_mask(s):
    mask = (torch.triu(torch.ones((s,s), device=DEVICE))==1).transpose(0,1) #s*s행렬을 1로 채우고, 상삼각행렬로 만들고 전치.
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask

#시퀀스를 입력받아 길이를 계산하고 마스크 생성 함수로 타깃 시퀀스의 마스크를 생성
def create_mask(src,tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len) #미래의 위치에 대해 -inf로 마스킹, 현재 및 이전 위치는 0으로 두어 계산에 포함.
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool) #모든 값을 False로 초기화하여 모든 토큰이 인코더의 셀프 어텐션 계산에 포함.

    src_padding_mask = (src == PAD_IDX).transpose(0,1) #패딩된 부분을 마스킹
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0,1) #패딩된 부분을 마스킹
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

target_input = target_tensor[:-1, :]
target_out = target_tensor[1:,:]

source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(
    source_tensor, target_input
)

# source_mask: 셀프 어텐션 과정에서 참조되는 소스 데이터의 시퀀스 범위
# False는 셀프 어텐션 참조 토큰, True는 제외되는 토큰
print("source_mask:", source_mask.shape)
print(source_mask)
# target_mask: [쿼리시퀀스길이, 키시퀀스길이]
print("target_mask:", target_mask.shape)
print(target_mask)
# 소스(타깃) 배치 데이터에서 텍스트 토큰이 존재하는지 여부
# False는 해당 토큰 존재, True는 해당 토큰이 패딩으로 채워짐
print("source_padding_mask:", source_padding_mask.shape)
print(source_padding_mask)
print("target_padding_mask:", target_padding_mask.shape)
print(target_padding_mask)

source_mask: torch.Size([35, 35])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
target_mask: torch.Size([29, 29])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
   

In [11]:
"""모델 학습 및 평가"""

def run(model, optimizer, criterion, split):
    model.train() if split == "train" else model.eval()
    data_iter = Multi30k(split=split,language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)

    losses = 0
    #소스와 타깃 데이터를 입력받고, collator로 문장을 토큰화 및 인덱스 변환
    for source_batch, target_batch in dataloader:
        source_batch = source_batch.to(DEVICE)
        target_batch = target_batch.to(DEVICE)

        #타깃 시퀀스를 입력과 출력으로 분리(<sos> A B C <eos>)
        target_input = target_batch[:-1,:] #<sos> A B C
        target_output = target_batch[1:,:] #A B C <eos>

        #결괏값들은 타깃 시퀀스의 i번째까지 토큰이 주어졌을 때 i+1번째 토큰을 예측하는 데 활용
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(
            source_batch, target_input
        )
        
        #Seq2SeqTransformer가 model
        logits = model(
            src=source_batch,
            trg=target_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_padding_mask=src_padding_mask,
            tgt_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask
        )

        optimizer.zero_grad()
        loss = criterion(logits.reshape(-1, logits.shape[-1]),target_output.reshape(-1))
        if split == "train":
            loss.backward()
            optimizer.step()
        losses += loss.item()

    return losses/len(list(dataloader))

for epoch in range(5):
    train_loss = run(model, optimizer, criterion, "train")
    val_loss = run(model, optimizer, criterion, "valid")
    print(f"Epoch:{epoch+1}, Train loss:{train_loss: .3f}, Val loss : {val_loss:.3f}")

/Users/yeeun/anaconda3/lib/python3.11/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch:1, Train loss: 4.855, Val loss : 4.126
Epoch:2, Train loss: 3.827, Val loss : 3.678
Epoch:3, Train loss: 3.497, Val loss : 3.553
Epoch:4, Train loss: 3.311, Val loss : 3.525
Epoch:5, Train loss: 3.177, Val loss : 3.473


In [12]:
"""트랜스포머 모델 변역 결과"""

#그리드 디코딩 방식으로 번역 결과 출력(현재 시점에서 가장 확률이 높은 단어를 선택하여 디코딩을 진행)
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    #소스 문자를 토큰 인덱스로 표현한 source_tensor를 생성하고, source_mask는 소스 문장에서 모든 토큰이 어텐션 될 수 있게 0으로 설정
    memory = model.encode(source_tensor, source_mask) #마지막 인코더 블록의 벡터 추출
    ys = torch.ones(1,1).fill_(start_symbol).type(torch.long).to(DEVICE) #타깃 데이터의 입력 텐서
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        target_mask = generate_square_subsequent_mask(ys.size(0))
        target_mask = target_mask.type(torch.bool).to(DEVICE)

        out = model.decode(ys, memory, target_mask) #[토큰 개수, 배치 크기, 확률]
        out = out.transpose(0,1) #[배치 크기, 토큰 개수, 확률]
        prob = model.generator(out[:,-1]) #[배치 크기, 확률]
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat(
            [ys, torch.ones(1,1).type_as(source_tensor.data).fill_(next_word)],dim=0
        )
        if next_word == EOS_IDX:
            break
    return ys

def translate(model, source_sentence):
    model.eval() #모델을 평가 모드로 설정
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence).view(-1,1) #소스 문장의 전처리
    num_tokens = source_tensor.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool) #소스 마스크 생성
    #그리디 디코딩으로 타깃 시퀀스를 생성
    tgt_tokens = greedy_decode( 
        model, source_tensor, src_mask, max_len=num_tokens +5, start_symbol=BOS_IDX
    ).flatten()
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1] #eos,bos토큰 제거
    return " ".join(output) #최종 번역문 반환

#번역 결과 출력
output_oov = translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")
output = translate(model, "Eine Gruppe von Menschen steht vor einem Gebäude .")
print(output_oov) #OOV 데이터로 인한 부정확한 결과 출력
print(output) #정확한 결과 출력

A group of people are standing outside a building .
A group of people are standing outside a building .
